<a href="https://colab.research.google.com/github/disimhot/DPO_ML_Spring2022/blob/main/HW_ML_ALL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
%matplotlib inline
import seaborn as sns

In [3]:
! pip install -q kaggle

In [8]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"disimhot","key":"50732584ebc7bf6745fa102eec47eec1"}'}

In [10]:
! chmod 600 ~/.kaggle/kaggle.json

In [12]:
! kaggle competitions download -c house-prices-advanced-regression-techniques

  0% 0.00/199k [00:00<?, ?B/s]
100% 199k/199k [00:00<00:00, 68.2MB/s]


In [14]:
! mkdir test

In [175]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

In [177]:
train.head(5).style.background_gradient(axis=None,cmap='YlOrRd')

train.dtypes.value_counts()

train.shape

train.isnull().sum().sort_values(ascending=False).head(20)

# plt.figure(figsize=(15,10))
# sns.heatmap(train.isnull(), yticklabels=False, cbar=False)

train['SalePrice'].describe()

# f, ax = plt.subplots(figsize=(8, 6))
# sns.distplot(train['SalePrice'])

# corrmat = train.corr()
# f, ax = plt.subplots(figsize=(15, 10))
# sns.heatmap(corrmat, vmax=.9, square=True);

k = 10
# cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
# cm = np.corrcoef(train[cols].values.T)
# sns.set(font_scale=1.25)
# hm = sns.heatmap(cm, cbar=True, annot=True, square=True, 
#                  fmt='.2f', annot_kws={'size': 10}, 
#                  yticklabels=cols.values, xticklabels=cols.values)
# plt.show()

Target = 'SalePrice'
train.dropna(axis=0, subset=[Target], inplace=True)

# Соединяем тренировочный и тестовый датасеты, чтобы провести преобразования на всех данных
# all_data = pd.concat([train.iloc[:,:-1], test],axis=0)

print('У тренировочного датасета {} рядов и {} признаков'.format(train.shape[0], train.shape[1]))
print('У тестового датасета {} рядов и {} признаков'.format(test.shape[0], test.shape[1]))

train.isnull().sum().sort_values(ascending=False).head(20)

#удалена ненужная колонка Id и колонки с макимальными пропусками
train.drop(['Id', 'PoolQC','Fence','MiscFeature','Alley','FireplaceQu'], axis=1,inplace=True) 
test.drop(['PoolQC','Fence','MiscFeature','Alley','FireplaceQu'], axis=1,inplace=True) 

train['LotFrontage'] = train['LotFrontage'].fillna(train['LotFrontage'].mean())
train['GarageQual'] = train['GarageQual'].fillna(train['GarageQual'].mode()[0])
train['GarageYrBlt'] = train['GarageYrBlt'].fillna(train['GarageYrBlt'].mode()[0])
train['GarageFinish'] = train['GarageFinish'].fillna(train['GarageFinish'].mode()[0])
train['GarageCond'] = train['GarageCond'].fillna(train['GarageCond'].mode()[0])
train['GarageType'] = train['GarageType'].fillna(train['GarageType'].mode()[0])

train.dropna(inplace=True)
test.dropna(inplace=True)

train.head()

У тренировочного датасета 1460 рядов и 81 признаков
У тестового датасета 1459 рядов и 80 признаков


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [178]:
# нужен для фин результата

test_id = test['Id']
# test.drop(['Id'], inplace=True)


In [182]:
test.drop('Id', axis=1, inplace=True)

In [183]:
y = train['SalePrice']
train.drop('SalePrice', axis=1, inplace=True)

In [184]:
train = train._get_numeric_data()
test = test._get_numeric_data()

ss = StandardScaler()
scaler = ss.fit(train)
train = scaler.fit_transform(train)
test = scaler.transform(test)

In [185]:
lr = LinearRegression()
lr.fit(train, y)

train_pred = lr.predict(train)
y_pred = lr.predict(test)

print("Train MSE: {}".format(np.sqrt(mean_squared_error(y, train_pred))))

print(f'''
train R2={r2_score(y, train_pred)}
''')

Train MSE: 34340.81626468771

train R2=0.8121952226696475



In [186]:
submission = pd.DataFrame({
    'Id': test_id,
    'SalePrice': y_pred
})

submission.head()

,Id,SalePrice
0,1461,114516.280209
1,1462,145625.812499
2,1463,171891.538040
3,1464,198902.711174
4,1465,194663.455851


In [187]:
submission.to_csv("submission.csv", index=False)